In [ ]:
pip install tensorflow


In [ ]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE=256
BATCH_SIZE=32
CHANNEL=3
EPOCHS=50


In [ ]:
# creating pipeline for reading images ,,,so that you can read it by batches 
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
    
)

In [ ]:
dir(dataset)

In [ ]:
class_names=dataset.class_names

In [ ]:
len(dataset)

In [ ]:
68*32   # so it will have more at the last batch

# Inspection on dataset

In [ ]:
for image_batch, label_batch in dataset.take(1):
    print(len(image_batch))   # each batch will be having 32 images
    print(image_batch.shape)  # shape of batch
    print(label_batch.numpy())  #target label for all 32 
    print(image_batch[0])    # it is a tensor
    print(image_batch[0].numpy()) # converted to numpy
    print(image_batch[0].shape)  #shape of each image
    plt.figure(figsize=(10,10))
    for i in range(12):
        ax=plt.subplot(4,3,i+1)  #we have created subplot so that all images can be seen
        plt.title(class_names[label_batch[i]])
        plt.imshow(image_batch[i].numpy().astype('uint8'))  #you have to use uint8 cz it waas float 
        plt.axis("off")
    
    

In [ ]:
# 80%==>training 
# 20%==>10% evaluation,10% test


In [ ]:
train_size=0.8
len(dataset)*0.8

# so 54 batch we will keep for training

In [ ]:
train_ds=dataset.take(54)
len(train_ds)

In [ ]:

test_ds=dataset.skip(54)
len(test_ds)



In [ ]:
val_size=0.1
len(dataset)*val_size

In [ ]:
val_ds=test_ds.take(6)

In [ ]:
len(val_ds)

In [ ]:
test_ds=test_ds.skip(6)
len(test_ds)

In [ ]:
# we will do the above splitting inside a function

def get_dataset_partitions_tf(ds,train_split=0.8,val_split=0.1,test_split=0.1,shuffle=True,shuffle_size=10000):
    ds_size=len(ds)
    if shuffle:
        ds=ds.shuffle(shuffle_size,seed=12)
    train_size=int(ds_size*train_split)
    val_size=int(ds_size*val_split)
    test_size=int(ds_size*test_split)
    train_ds=ds.take(train_size)
    val_ds=ds.skip(train_size).take(val_size)
    test_size=ds.skip(train_size).skip(val_size)
    return train_ds,val_ds,test_ds

In [ ]:
train_ds,val_ds,test_ds=get_dataset_partitions_tf(dataset)

In [ ]:
len(train_ds)

In [ ]:
len(val_ds)

In [ ]:
len(test_ds)

In [ ]:
# this will keep the batches in cache memeory...so that you dont need to load again and again
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# will do the preprocessing 

resize_and_rescale=tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
    
])

In [ ]:
data_augmentation=tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
input_shape=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNEL)
n_classes=3
model=models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D(pool_size=(2, 2)),
     layers.Conv2D(64,(3,3),activation='relu',),
    layers.MaxPooling2D(pool_size=(2, 2)),
     layers.Conv2D(64,(3,3),activation='relu',),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax'),
    
])
model.build(input_shape)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
             metrics=['accuracy'])

In [ ]:
history=model.fit(
          train_ds,
        epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
          )

In [ ]:
scores=model.evaluate(test_ds)

In [ ]:
history


In [ ]:
# you can see the documentation of keras.src.callbacks
history.params

In [ ]:
history.history.keys()

In [ ]:
history.history['accuracy']

In [ ]:
len(history.history['accuracy'])

In [ ]:
# will do the visualization
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_accuracy']

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS),acc,label="Training Accuracy")
plt.plot(range(EPOCHS),val_acc,label="validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and validation Accuarcy")


plt.subplot(1,2,2)
plt.plot(range(EPOCHS),loss,label="Training Loss")
plt.plot(range(EPOCHS),val_loss,label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")
plt.show()


In [ ]:
# testing
for images_batch,labels_batch in test_ds.take(1):
    print(images_batch[0].numpy())  #prints value
    
    first_image=plt.matshow(images_batch[0].numpy().astype('uint8'))
    first_label=
    